In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mybiotools as mbt
import pr_peaks

# 2018-02-22 Traffic
In the chair model there is a quantity that I haven't explored so far, which is the traffic in- and out-going from/to the chairs. In the flat transition matrix model, there should not be any difference (normalized to the tau of the chair) between the number of out-going and in-going events. However, in the non-uniform transition matrix model, I expect that there will be some interesting features going on.

In [ ]:
# prepare site_taus
n = 100
nH = 10
np.random.seed(6589)
# Hsites and Lsites
Hsites = np.random.choice(n,size=nH,replace=False)
Lsites = range(n)
for site in Hsites :
    Lsites.remove(site)
site_taus = 2.0*np.ones(n)
site_taus[Hsites] = 20.0
# plot
fig = plt.figure(figsize=(10,3))
ax = plt.subplot(111)
mbt.line_plot(ax,np.arange(n),site_taus)
plt.show()

In [ ]:
# do the simulations
N = 100
nsteps = 1000
Tflat = np.ones((n,n))/n
flat = pr_peaks.JumpingModel(Tflat,site_taus)
mus = range(1,10)
for mu in mus :
    mbt.log_message('Flat','mu = %d'%(mu))
    omega_t_initial = pr_peaks.init_omega_t(N,n,mu)
    flat.run(nsteps,mu,sigma=None,omega_t_initial=omega_t_initial)
pr_peaks.H_to_L(flat,Hsites,Lsites)

In [ ]:
THL = np.ones((n,n))
for Hsite in Hsites :
    for Lsite in Lsites :
        THL[Hsite,Lsite] += 1
        THL[Lsite,Hsite] += 1
THL = mbt.row_normalize_matrix(THL)
HL = pr_peaks.JumpingModel(THL,site_taus)
for mu in mus :
    mbt.log_message('HL','mu = %d'%(mu))
    omega_t_initial = pr_peaks.init_omega_t(N,n,mu)
    HL.run(nsteps,mu,sigma=None,omega_t_initial=omega_t_initial)
pr_peaks.H_to_L(HL,Hsites,Lsites)

In [ ]:
plt.plot(mus,flat.H_to_L,'r')
plt.plot(mus,HL.H_to_L,'b')
plt.xlabel(r'$\mu$')
plt.ylabel('H to L ratio')
plt.axhline(y=1.0,linestyle='--',color='k')
plt.show()

Okay this result is the familiar ambiguous one: in some cases I obtained a maximum h-enhancement for intermediate values of $\mu$, in others I didn't. But now let's look at the traffic.

In [ ]:
mu = 5
fig = plt.figure(figsize=(10,13))
gs = plt.GridSpec(2,1,hspace=0,height_ratios=[10,3])
ax = plt.subplot(gs[0,0])
ax.matshow(flat.J[mu])
ax = plt.subplot(gs[1,0],sharex=ax)
y = flat.J[mu].sum(axis=1)
mbt.line_plot(ax,Hsites,y[Hsites],color='r')
mbt.line_plot(ax,Lsites,y[Lsites],color='b')
ax.set_ylim(min(y)-100,max(y)+100)
plt.show()

I want to verify that some identities concerning the jump matrix are verified numerically.

In [ ]:
mu = 1
x = np.arange(n)
j_out = flat.J[mu].sum(axis=1)
j_out_theory = nsteps * flat.occupancy[mu]/site_taus
fig = plt.figure(figsize=(10,4))
ax = plt.subplot(211)
mbt.line_plot(ax,Lsites,j_out[Lsites],color='b')
mbt.line_plot(ax,Hsites,j_out[Hsites],color='r')
delta = j_out.max()-j_out.min()
ax.set_ylim(j_out.min()-delta/10.,j_out.max()+delta/10.)
ax = plt.subplot(212)
mbt.line_plot(ax,x,j_out_theory,color='r')

This analysis showed me that it is very important to obtain good statistics from the simulations, otherwise all the simulation results will be ambiguous. I should consider rewriting the simulation code to simulate a system composed of only one system, but performing time averages. To have an idea of how many time points I need to take, I should evaluate the autocorrelation of the configurations of the system.

## Rewriting simulation code

First, let's rewrite the simulation main loop, taking into account that the system is now one.

**NOTE**: the above code won't work anymore! To make it work, revert to commit

3ec9d25bce3b693

in pr_peaks

In [ ]:
# do the simulations with the new code implemented
n = 100
nsteps = 100000
Tflat = np.ones((n,n))/n
flat = pr_peaks.JumpingModel(Tflat,site_taus)
mu = 1
mbt.log_message('Flat','Start')
omega_t_initial = pr_peaks.init_omega_t(n,mu)
flat.run(nsteps,mu,None,omega_t_initial,teq=1000,tsample=10)
mbt.log_message('Flat','End')

Now that we have the new code up and running, I can do the autocorrelation analysis.

In [ ]:
nsamples = flat.samples[mu].shape[0]
ac = np.zeros(nsamples)
b = flat.samples[mu][0,:].astype(float)
for sample in xrange(nsamples) :
    a = flat.samples[mu][sample,:].astype(float)
    ac[sample] = np.dot(a,b)/mu

In [ ]:
plt.plot(mbt.autocorrelation(ac))
plt.show()

In [ ]:
plt.bar(np.arange(n),flat.theta[mu])
plt.show()

Okay, this seems to be working correctly. Now it's time to try to do statistics correctly, and figure out a way of implementing reasonable traffic analysis.

In [ ]:
# general simulation parameters
nsteps = 101000
teq = 1000
tsample = 1
sigma=None
mus = range(1,10)
deterministic=True

# prepare site_taus
n = 100
nH = 10
np.random.seed(6589)
# Hsites and Lsites
Hsites = np.random.choice(n,size=nH,replace=False)
Lsites = range(n)
for site in Hsites :
    Lsites.remove(site)
site_taus = 2.0*np.ones(n)
site_taus[Hsites] = 20.0

# FLAT transition matrix
Tflat = np.ones((n,n))
for i in xrange(n) :
    Tflat[i,i] = 0.0
Tflat = mbt.row_normalize_matrix(Tflat)
flat = pr_peaks.JumpingModel(Tflat,site_taus)

# HL model
THL = np.ones((n,n))
for i in xrange(n) :
    THL[i,i] = 0.0
for Lsite in Lsites :
    for Hsite in Hsites :
        THL[Lsite,Hsite] += 2
THL = mbt.row_normalize_matrix(THL)
HL = pr_peaks.JumpingModel(THL,site_taus)

# simulations
for mu in mus :
    omega_t_initial = pr_peaks.init_omega_t(n,mu)
    mbt.log_message('Flat','mu = %d'%(mu))
    flat.run(nsteps,mu,sigma,omega_t_initial,
            teq=teq,tsample=tsample,deterministic=deterministic)
    mbt.log_message('HL','mu = %d'%(mu))
    HL.run(nsteps,mu,sigma,omega_t_initial,
            teq=teq,tsample=tsample,deterministic=deterministic)

# aftermath
pr_peaks.H_to_L(flat,Hsites,Lsites)
pr_peaks.H_to_L(HL,Hsites,Lsites)

In [ ]:
plt.plot(mus,flat.H_to_L,'r',linewidth=3,label='Flat model')
plt.plot(mus,HL.H_to_L,'b',linewidth=3,label='HL model')
plt.legend(loc='upper right')
plt.xlabel('Number of searchers')
plt.ylabel('H to L ratio')
plt.show()

I don't understand why there is this number coming out of the simulations. I expect that the ratio at one searcher will be equal to the ratio of the $\tau$ values of the high and low affinity sites. Let's try with 2 sites and one searcher.

Okay, I fixed it, at least for the deterministic case.